In [1]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1


def get_game_info(thisgame_df, row):
    
    thisgame_df['visteam'] = row['info']['visteam']
    thisgame_df['hometeam'] = row['info']['hometeam']
    thisgame_df['date'] = pandas.to_datetime(row['info']['date'], errors='coerce')
    thisgame_df['time'] = row['info']['starttime']
    thisgame_df['daynight'] = row['info']['daynight']
    thisgame_df['site'] = row['info']['site']
    thisgame_df['length'] = pandas.to_timedelta(str(row['info']['timeofgame'])+' minutes', errors='ignore')
    thisgame_df['attendance'] = pandas.to_numeric(row['info']['attendance'])
    thisgame_df['fieldcond'] = row['info']['fieldcond']
    thisgame_df['precip'] = row['info']['precip']    
    thisgame_df['sky'] = row['info']['sky']
    thisgame_df['winddir'] = row['info']['winddir']
    thisgame_df['windspeed'] = pandas.to_numeric(row['info']['windspeed'])
    return thisgame_df

def parse_events(thisgame_df):
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['modifiers'] = thisgame_df['modifiers'].fillna('')
    thisgame_df['baserunning'] = thisgame_df['modifiers'].str.split('.',n=1).str.get(1)
    thisgame_df['baserunning'] = thisgame_df['baserunning'].fillna('')
    thisgame_df['modifiers'] = thisgame_df['modifiers'].str.split('.').str.get(0)
#    thisgame_df['mdfs'] = thisgame_df['ev'].str.split('/',n=1).str.get(1)
#    thisgame_df['mdfs'] = thisgame_df['mdfs'].fillna('')
#    thisgame_df['brun'] = thisgame_df['mdfs'].str.split('.',n=1).str.get(1)
#    thisgame_df['brun'] = thisgame_df['brun'].fillna('')
#    thisgame_df['mdfs'] = thisgame_df['mdfs'].str.split('.').str.get(0)
    
    return thisgame_df

In [2]:
# runs in about 5 minutes
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

ok


In [ ]:
#allgames_df.sample(10).index
#'MIN201204230', 'SEA201207290', 'LAN201304280', 'OAK201309010',
#       'SFN201409270', 'DET201306090', 'TEX201208240', 'ANA201404280',
#       'WAS201304260', 'CIN201609060'],

In [3]:
#allgames_df = allgames_df.iloc[allgames_df.index == 'MIN201204230']
#allgames_df = allgames_df.sample(1)

column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']
#column_list = ['eig', 'et', 'inn', 'hv', 'bID']
#column_list += ['pc', 'ps', 'ev', 'com', 'sbord', 'spos']

#showcolumns = ['gameID']
#showcolumns += ['visteam', 'hometeam','site', 'date', 'time', 'length']
#showcolumns = ['et', 'inn', 'hv', 'bID', 'pID']
#showcolumns += ['pc', 'ps']
#showcolumns += ['pc', 'ev', 'play', 'mdfs', 'brun', 'com', 'sbord', 'spos']
#showcolumns += ['event_type', 'inning', 'home_or_visitor']
#showcolumns += ['batterID', 'event', 'play']
#showcolumns += ['modifiers', 'baserunning']
#showcolumns += ['comment', 'sub_batting_order', 'sub_position']

In [4]:
allevents_df = pandas.DataFrame(columns = column_list)

cnt = 0
for thisgameID,row in allgames_df.iterrows():
    if (numpy.mod(cnt,100) == 0):
        print('Parsing game {0:,.0f}: {1}'.format(cnt, thisgameID))
    thisgame_df = pandas.DataFrame(row['events'], columns=column_list)
    thisgame_df = thisgame_df.set_index('event_in_game')

    thisgame_df['gameID'] = thisgameID
    
    thisgame_df = get_game_info(thisgame_df, row)

#    visiting_roster_df = pandas.DataFrame(row['visiting_starters'], columns=['playerID', 'playerName', 'batting_order', 'position'])
#    home_roster_df = pandas.DataFrame(row['home_starters'], columns=['playerID', 'playerName', 'batting_order', 'position']) 
#    visiting_roster_df['hv'] = 0
#    home_roster_df['hv'] = 1

#    playing_df = pandas.concat((visiting_roster_df, home_roster_df))
#    playing_df['batting_order'] = pandas.to_numeric(playing_df['batting_order'])
#    playing_df['position'] = pandas.to_numeric(playing_df['position']) 

    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df = parse_events(thisgame_df)

    allevents_df = pandas.concat((allevents_df, thisgame_df))
    
    cnt = cnt + 1

print('Done')

allevents_df.to_csv('events_2012-2016_parsed.json')

Parsing game 0: ANA201204060
Parsing game 100: ARI201205210
Parsing game 200: ATL201207020
Parsing game 300: BAL201208120
Parsing game 400: BOS201209210
Parsing game 500: CHN201205050
Parsing game 600: CIN201206220
Parsing game 700: CLE201208060
Parsing game 800: COL201209100
Parsing game 900: HOU201204300
Parsing game 1,000: KCA201206120
Parsing game 1,100: LAN201207170
Parsing game 1,200: MIA201209040
Parsing game 1,300: NYA201204170
Parsing game 1,400: NYN201205270
Parsing game 1,500: OAK201207070
Parsing game 1,600: PHI201208220
Parsing game 1,700: PIT201210030
Parsing game 1,800: SEA201205240
Parsing game 1,900: SFN201206270
Parsing game 2,000: TBA201208070
Parsing game 2,100: TEX201209250
Parsing game 2,200: WAS201205040
Parsing game 2,300: ANA201306050
Parsing game 2,400: ARI201307260
Parsing game 2,500: ATL201309040
Parsing game 2,600: BOS201304212


KeyboardInterrupt: 